In [1]:
import os
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import spacy
import nltk
from nltk.corpus import wordnet

In [2]:
os.chdir(os.path.dirname(os.getcwd()))
rootdir=os.getcwd()+'/dataset/' 
directories=[]
for file in os.listdir(rootdir):
    d = os.path.join(rootdir, file)
    if os.path.isdir(d):
        directories.append(d)

In [12]:
datas={}
nlp = spacy.load("en_core_web_sm")
import neuralcoref
neuralcoref.add_to_pipe(nlp)
nlp_ = spacy.load("en_core_web_sm")
for i in directories:
    pref=str(i.split('/')[-1])
    files = [f for f in listdir(i) if isfile(join(i, f))]
    for f in files:
        if f[-4:]=='.txt':
            suf=f[:-4]
            name=pref+suf
            with open(i+'/'+f, "r") as file:
                data = file.read().replace("\n", "")
            article = data.replace(u"\xa0", u" ")
            datas[name]=nlp(article)
        

In [4]:
datas['reuters2ea97a0695a79edc852ba9565bfe7220']._.coref_resolved

'America Movil dips on pay TV permit snag, minor impact seenThe logo of America Movil is pictured on the wall at a reception area in America Movil\'s corporate offices, in Mexico City, Mexico January 25, 2022. REUTERS/Gustavo Graf/File PhotoMEXICO CITY, Jan 27 (Reuters) - Shares of Mexican telecoms giant America Movil fell on Thursday after a regulator scuttled America Movil long-term goal to enter the local pay TV market, raising speculation over the next moves by the company controlled by the family of Mexican tycoon Carlos Slim.The stock dropped 3.94%, the biggest daily dip since September, exceeding the 1% decline in Mexico\'s S&P/BMV IPC (.MXX), but analysts noted even broader losses on financial markets.The Mexican telecom industry regulator, the Federal Telecommunications Institute (IFT), concluded a vote on Wednesday without authorizing America Movil a pay TV license in that country, a source with knowledge of the process told REUTERS/.REUTERS/ is already Latin America\'s large

In [11]:
datas['reuters2ea97a0695a79edc852ba9565bfe7220']._.coref_clusters

[America Movil: [America Movil, America Movil, the company, America Movil, its, America Movil, America Movil, Movil, America Movil, America Movil, America Movil, America Movil's, America Movil, America Movil, America Movil],
 Mexico: [Mexico, Mexico],
 REUTERS/: [REUTERS/, Reuters, The company],
 Mexican: [Mexican, Mexican],
 Carlos Slim: [Carlos Slim, Carlos Slim, He],
 America: [America, America],
 the IFT: [the IFT, its],
 U.S.: [U.S., U.S.],
 Reuters: [Reuters, The firm, its, it, the company],
 America Movil executives: [America Movil executives, they, their],
 he: [he, he],
 President Andres Manuel Lopez Obrador: [President Andres Manuel Lopez Obrador, Lopez Obrador],
 The tycoon's construction arm Grupo Carso: [The tycoon's construction arm Grupo Carso, its, Carso]]

In [5]:
datas['reuters2ea97a0695a79edc852ba9565bfe7220']

America Movil dips on pay TV permit snag, minor impact seenThe logo of America Movil is pictured on the wall at a reception area in the company's corporate offices, in Mexico City, Mexico January 25, 2022. REUTERS/Gustavo Graf/File PhotoMEXICO CITY, Jan 27 (Reuters) - Shares of Mexican telecoms giant America Movil fell on Thursday after a regulator scuttled its long-term goal to enter the local pay TV market, raising speculation over the next moves by the company controlled by the family of Mexican tycoon Carlos Slim.The stock dropped 3.94%, the biggest daily dip since September, exceeding the 1% decline in Mexico's S&P/BMV IPC (.MXX), but analysts noted even broader losses on financial markets.The Mexican telecom industry regulator, the Federal Telecommunications Institute (IFT), concluded a vote on Wednesday without authorizing America Movil a pay TV license in that country, a source with knowledge of the process told Reuters.The company is already Latin America's largest pay TV prov

In [6]:
text_names = [f for f in datas.keys()]

In [7]:
def list_entities(text):
    ents_=text.ents
    ents=[]
    for x in ents_:
        if x.text not in ents:
            if x.label_=="ORG" or x.label_=="PRODUCT" or x.label_=="GPE":
                ents.append(x.text)
    return ents


def find_org(s, ents):
    i=0
    ind=[]
    for token in s:
        if str(token) in ents: 
            ind.append(i)
        i=i+1
    return ind

def check_synonym_SVO(s, list_):
    for token in s:
        if token.dep_=="ROOT":
            lem=token.lemma_
            if lem in list_:
                return True
            else:
                return False      

def find_sbj(s, ind):
    subjects=[]
    for i in ind:
        if s[i].dep_=="nsubj":
            subjects.append(s[i])
        elif s[i].head.dep_=="nsubj": # walk one step up the dependency tree 
            subjects.append(s[i])
        else:
            continue # could move one more step up the dependency tree if we condition was came up before (excluding object and verbs)
    return subjects

def find_nsubjpass(s, ind):
    subjects=[]
    for i in ind:
        if s[i].dep_=="nsubjpass":
            subjects.append(s[i])
        elif s[i].head.dep_=="nsubjpass": # walk one step up the dependency tree 
            subjects.append(s[i])
        else:
            continue # could move one more step up the dependency tree if we condition was came up before (excluding object and verbs)
    return subjects
            
def find_obj(s, ind):
    subjects=[]
    for i in ind:
        if s[i].dep_=="dobj" or s[i].dep_=="pobj":
            subjects.append(s[i])
        elif s[i].head.dep_=="dobj" or s[i].head.dep_=="pobj": # walk one step up the dependency tree 
            subjects.append(s[i])
        else:
            continue
    return subjects

def check_synonym_attr(s, list_):
    for token in s:
        if token.dep_=="attr":
            lem=token.lemma_
            if lem in list_:
                return True
            else:
                return False
            
def find_obj_attr(s, ind):
    subjects=[]
    for i in ind:
        if s[i].dep_=="pobj":
            subjects.append(s[i])
        elif s[i].head.dep_=="pobj": # walk one step up the dependency tree 
            subjects.append(s[i])
        else:
            continue
    return subjects

def check_synonym_acl(s, list_):
    for token in s:
        if token.dep_=="acl":
            lem=token.lemma_
            if lem in list_:
                return True
            else:
                return False  

def find_obj_acl(s, ind):
    subjects=[]
    for i in ind:
        if s[i].dep_=="pobj":
            subjects.append(s[i])
        elif s[i].head.dep_=="pobj": # walk one step up the dependency tree 
            subjects.append(s[i])
        else:
            continue
    return subjects

# SVO

Get synonyms

In [8]:
synonyms = []
antonyms = []
  
for syn in wordnet.synsets("compete"):
    for l in syn.lemmas():
        synonyms.append(l.name())
        if l.antonyms():
            antonyms.append(l.antonyms()[0].name())
print(synonyms)
print(antonyms)

['compete', 'vie', 'contend']
[]


In [13]:
tuples=[]
relation="competitors"
synonyms=["compete","challenge","clash"] # have is too misleading
for i in text_names:
    text=datas[i]._.coref_resolved
    text=nlp_(text)
    ents=list_entities(text)
    for s in text.sents:
        dict_={}
        org=find_org(s,ents)
        if len(org)>=2:
            SVO=check_synonym_SVO(s,synonyms)
            if SVO==True:
                sbj=find_sbj(s,org)
                if len(sbj)>0: 
                    obj=find_obj(s,org)
                    if len(obj)>0:
                        dict_['relation']=relation
                        ## add ID later dict_['relation']
                        dict_['sentence']=s
                        dict_['entities_A']=sbj
                        dict_['entities_B']=obj
                        tuples.append(dict_)
                else:
                    nsubjpass=find_nsubjpass(s,org)
                    if len(nsubjpass)>0:
                        obj=find_obj(s,org)
                        if len(obj)>0:
                            dict_['relation']=relation
                            ## add ID later dict_['relation']
                            dict_['sentence']=s
                            dict_['entities_A']=obj
                            dict_['entities_B']=sbj
                            tuples.append(dict_)
                    else:
                        continue
            else:
                continue
        else:
            continue    

In [14]:
len(tuples)

0

In [15]:
tuples

[]

# attributes kind of patterns

In [16]:
synonyms = []
antonyms = []
  
for syn in wordnet.synsets("competitors"):
    for l in syn.lemmas():
        synonyms.append(l.name())
        if l.antonyms():
            antonyms.append(l.antonyms()[0].name())
print(synonyms)
print(antonyms)

['rival', 'challenger', 'competitor', 'competition', 'contender']
[]


In [17]:
tuples=[]
relation="competitors"  
synonyms=["competitor","rival","challenger","contender"]
for i in text_names:
    text=datas[i]._.coref_resolved
    text=nlp_(text)
    ents=list_entities(text)
    for s in text.sents:
        dict_={}
        org=find_org(s,ents)
        if len(org)>=2:
            attr=check_synonym_attr(s,synonyms)
            if attr==True:
                sbj=find_sbj(s,org)
                if len(sbj)>0: 
                    obj=find_obj_attr(s,org)
                    if len(obj)>0:
                        dict_['relation']=relation
                        ## add ID later dict_['relation']
                        dict_['sentence']=s
                        dict_['entities_A']=sbj
                        dict_['entities_B']=obj
                        tuples.append(dict_)

In [18]:
tuples

[]

# ACL based

In [19]:
tuples=[]
relation="ownership"
synonyms=["compete","challenge","clash"] # have is too misleading
for i in text_names:
    text=datas[i]._.coref_resolved
    text=nlp_(text)
    ents=list_entities(text)
    for s in text.sents:
        dict_={}
        org=find_org(s,ents)
        if len(org)>=2:
            acl=check_synonym_acl(s,synonyms)
            if acl==True:
                sbj=find_sbj(s,org)
                if len(sbj)>0: 
                    obj=find_obj_acl(s,org)
                    if len(obj)>0:
                        dict_['relation']=relation
                        ## add ID later dict_['relation']
                        dict_['sentence']=s
                        dict_['entities_A']=sbj
                        dict_['entities_B']=obj
                        tuples.append(dict_)

In [20]:
tuples

[]